In [2]:
from vnpy_portfoliostrategy.mysqlservice import MysqlService

In [3]:
from datetime import datetime
import pandas as pd
import numpy as np

In [59]:
db = MysqlService()

current_day = datetime(2024,9,9)
rebal_tar = db.select('daily_rebalance_target',today = current_day, strategy = 'strategy2')
# trading_schedule = db.select('trading_schedule',today = current_day, strategy = 'strategy2').set_index('id').drop_duplicates()
# previous_trading_schedule = db.select('trading_schedule',date = current_day, strategy = 'strategy2').set_index('id').drop_duplicates()
# trading_hours = db.select('trading_hours',date = trading_schedule['date'].iloc[0])

# if trading_schedule.shape[0]!=1 or previous_trading_schedule.shape[0]!=1:
#     raise Exception('Wrong trading schedule for strategy2')

db.close()


c:\veighna_studio\lib\site-packages\vnpy_portfoliostrategy\mysqlservice.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(f"SELECT * FROM `vnpy`.`{table}` {'where' if len(where)>0 else ''} {self.dict_to_string(where)}" + additional_query, self.mydb)


In [60]:
rebal_tar = pd.concat([pd.Series(rebal_tar['symbol'].values[0].split(',')),
                        pd.Series(rebal_tar['target'].values[0].split(','))],axis=1,keys=['symbol','target'])

In [63]:
pd.concat([pd.Series(trading_schedule['symbol'].values[0].split(',')).str[:-4],
           pd.Series(trading_schedule['symbol'].values[0].split(','))],axis=1,
          keys=['symbol','symb']).merge(trading_hours[['rqsymbol','symbol']],
                                        left_on='symb',right_on='rqsymbol',how='inner').merge(rebal_tar,
                                                                                                left_on='symbol_x',right_on='symbol',how='inner')[['symbol_y','target']]

,symbol_y,target
0,a2501.DCE,-1
1,al2410.SHFE,1
2,ao2411.SHFE,1
3,AP501.CZCE,0
4,b2410.DCE,0
5,bc2411.INE,0
6,br2411.SHFE,-1
7,bu2411.SHFE,0
8,c2411.DCE,0
9,CF501.CZCE,0
